In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv('Munich House raw.csv',delimiter='\t')
data.head()

,index0,terrace,bedrooms,construction_year,date_month,date_year,date_year_month,floor,rooms,price_room,price_surface,price,surface,zip
0,0,yes,2,1986,3,2016,2016_03,up floor,3,108333.33333333333,3915.6626506024095,325000,83,85221
1,1,yes,NAN,1972,3,2016,2016_03,up floor,3,115000.0,4107.142857142857,345000,84,85521
2,2,yes,2,1968,3,2016,2016_03,up floor,3,123333.33333333333,4805.194805194805,370000,77,81369
3,3,yes,2,1963,3,2016,2016_03,up floor,3,133000.0,4244.68085106383,399000,94,82110
4,4,yes,2,1973,3,2016,2016_03,up floor,3,122933.33333333333,4789.61038961039,368800,77,81735


In [3]:
data=data.drop('index0',1)

In [4]:
data.head()

,terrace,bedrooms,construction_year,date_month,date_year,date_year_month,floor,rooms,price_room,price_surface,price,surface,zip
0,yes,2,1986,3,2016,2016_03,up floor,3,108333.33333333333,3915.6626506024095,325000,83,85221
1,yes,NAN,1972,3,2016,2016_03,up floor,3,115000.0,4107.142857142857,345000,84,85521
2,yes,2,1968,3,2016,2016_03,up floor,3,123333.33333333333,4805.194805194805,370000,77,81369
3,yes,2,1963,3,2016,2016_03,up floor,3,133000.0,4244.68085106383,399000,94,82110
4,yes,2,1973,3,2016,2016_03,up floor,3,122933.33333333333,4789.61038961039,368800,77,81735


In [5]:
data.dtypes

terrace              object
bedrooms             object
construction_year    object
date_month            int64
date_year             int64
date_year_month      object
floor                object
rooms                object
price_room           object
price_surface        object
price                object
surface              object
zip                   int64
dtype: object

In [6]:
data['terrace'].isna().sum()
# these NAN are string NAN not np.nan
# for handling nan values, first we have to convert them into np.nan

0

In [7]:
data.replace('NAN', np.nan, inplace=True)

In [8]:
data.isna().sum()
# now we have missing data to handle

terrace                0
bedrooms             319
construction_year     49
date_month             0
date_year              0
date_year_month        0
floor                113
rooms                 34
price_room            35
price_surface        212
price                  1
surface              211
zip                    0
dtype: int64

In [9]:
# columns having % missing data
(data.isna().sum()/len(data))*100

terrace               0.000000
bedrooms             36.708861
construction_year     5.638665
date_month            0.000000
date_year             0.000000
date_year_month       0.000000
floor                13.003452
rooms                 3.912543
price_room            4.027618
price_surface        24.395857
price                 0.115075
surface              24.280783
zip                   0.000000
dtype: float64

In [10]:
#remove the missing field of price, it is gonna our target feature
data[data['price'].isna()]

,terrace,bedrooms,construction_year,date_month,date_year,date_year_month,floor,rooms,price_room,price_surface,price,surface,zip
367,yes,NaN,2016,8,2016,2016_08,NaN,5,NaN,NaN,NaN,209,81479


In [11]:
data.drop(data[data['price'].isna()].index, inplace=True)
#remove using index (entire row)

In [12]:
# changing columns to their dtype based on records data type they having
for i in data.columns: 
    if data[i].dtypes=='object':
        try:
            data[i]=data[i].astype(float)
        except:
            data[i]=data[i].astype(str)

In [13]:
data.dtypes

terrace               object
bedrooms             float64
construction_year    float64
date_month             int64
date_year              int64
date_year_month      float64
floor                 object
rooms                float64
price_room           float64
price_surface        float64
price                float64
surface              float64
zip                    int64
dtype: object

In [14]:
data['floor'].unique()

array(['up floor', 'nan', 'ground floor'], dtype=object)

In [15]:
def floor_fill():
    if np.random.randint(0,8)==0:
        return 'up floor'
    else:
        return 'ground floor'
# fill the missing with random data

In [16]:
for i, value in data.iterrows():
    if data.loc[i,'floor']=='nan':
        data.loc[i,'floor']=floor_fill()

In [17]:
data['floor'].unique()

array(['up floor', 'ground floor'], dtype=object)

In [18]:
data['floor'].isna().sum()
# no missing values in floor

0

In [19]:
data.head()

,terrace,bedrooms,construction_year,date_month,date_year,date_year_month,floor,rooms,price_room,price_surface,price,surface,zip
0,yes,2.0,1986.0,3,2016,201603.0,up floor,3.0,108333.333333,3915.662651,325000.0,83.0,85221
1,yes,NaN,1972.0,3,2016,201603.0,up floor,3.0,115000.000000,4107.142857,345000.0,84.0,85521
2,yes,2.0,1968.0,3,2016,201603.0,up floor,3.0,123333.333333,4805.194805,370000.0,77.0,81369
3,yes,2.0,1963.0,3,2016,201603.0,up floor,3.0,133000.000000,4244.680851,399000.0,94.0,82110
4,yes,2.0,1973.0,3,2016,201603.0,up floor,3.0,122933.333333,4789.610390,368800.0,77.0,81735


In [20]:
conditions=[(data['rooms'].isnull()) & (data['bedrooms'].isnull()) & (data['surface'].isnull()),
             (data['rooms'].isnull()) & (data['bedrooms'].isnull()) & (data['surface']>75),
             (data['rooms'].isnull()) & (data['bedrooms'].notnull()),]
choices = [2,3,data['bedrooms']+1]
data['rooms'] = np.select(conditions,choices,default=2)
## randomly fill missing values for rooms based on bedrooms we are having

In [21]:
data['rooms'].value_counts()

2.0    836
3.0     24
4.0      8
Name: rooms, dtype: int64

In [22]:
data['bedrooms']=np.where(data['bedrooms'].isna(), data['rooms']-1, data['bedrooms'])
# filling bedrooms with 1room lesser than rooms available

In [23]:
data.isna().sum()

terrace                0
bedrooms               0
construction_year     49
date_month             0
date_year              0
date_year_month        0
floor                  0
rooms                  0
price_room            34
price_surface        211
price                  0
surface              211
zip                    0
dtype: int64

In [24]:
data.groupby('rooms')['surface'].mean()

rooms
2.0     89.988924
3.0     91.100000
4.0    100.400000
Name: surface, dtype: float64

In [25]:
## fill surface based on other romm's surface's mean
conditions=[(data['rooms']==2)&(data['surface'].isna()),
           (data['rooms']==3)&(data['surface'].isna()),
            (data['rooms']==4)&(data['surface'].isna())
           ]
choices=[90,91.1,100]
data['surface']=np.select(conditions, choices, default=90.5)

In [26]:
# fill the construction_year with 
data['construction_year'].fillna(data['construction_year'].mean()//1, inplace=True)

In [27]:
np.argwhere(data['price']==0)

C:\Users\aakash\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


array([[ 57],
       [373],
       [408],
       [549],
       [599],
       [662],
       [720],
       [731],
       [783],
       [804],
       [815],
       [817],
       [847],
       [867]], dtype=int64)

In [28]:
data.drop(data[[i==0 for i in data['price']]].index, inplace=True)
# target feature having zero, remove them

In [29]:
data['price_room'] = data['price'] / data['rooms']
data['price_surface'] = data['price'] / data['surface']

In [30]:
data.drop_duplicates(keep='first', inplace=True)
#remove duplicate data

### convert to cleaned file

In [31]:
data.to_csv('Munich_House_clean.csv', index=False)